In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
var('xx')
Q.<i> = NumberField(xx^2 + 1)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [ ]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

In [ ]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

In [5]:
with open('all_L_sets.pickle', 'rb') as fil:
    all_L_sets = pickle.load(fil)

In [6]:
L_set_base = ('E1', 'G4', 'E2', 'G3', 'E3')
possible_L_sets = [L_set for L_set in all_L_sets if L_set[0:3] == ('E1', 'G4', 'E3')]
possible_projs = [SE1.find_projectivity(L_set_base, L_set) for L_set in possible_L_sets]

In [8]:
%%time 
L_set_ideals = []
all_ideals = []
for L_set in possible_L_sets:
    proj = SE1.find_projectivity(L_set_base, L_set)
    ideals = SE1.find_decomposed_conditions_on_cubic(proj)
    ideals = [ide for ide in ideals if SE1.is_ideal_valid(ide)]
    all_ideals += ideals
    L_set_ideals.append([L_set, ideals])

CPU times: user 45.8 s, sys: 79.9 ms, total: 45.8 s
Wall time: 45.9 s


In [9]:
L_set_ideals

[[('E1', 'G4', 'E3', 'G6', 'E6'), []],
 [('E1', 'G4', 'E3', 'G6', 'F45'), []],
 [('E1', 'G4', 'E3', 'G6', 'F24'),
  [Ideal (4*c + (i)*d + (-i)*e + (i + 2)*f, 5*d^2 + (4*i - 2)*d*e + (-4*i - 3)*e^2 + (-8*i - 6)*d*f + (-36*i + 38)*e*f + (24*i - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1,
   Ideal (4*c + (-i)*d + (i)*e + (-i + 2)*f, 5*d^2 + (-4*i - 2)*d*e + (4*i - 3)*e^2 + (8*i - 6)*d*f + (36*i + 38)*e*f + (-24*i - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1]],
 [('E1', 'G4', 'E3', 'F13', 'E6'), []],
 [('E1', 'G4', 'E3', 'F13', 'E5'), []],
 [('E1', 'G4', 'E3', 'F13', 'E2'), []],
 [('E1', 'G4', 'E3', 'G5', 'F46'), []],
 [('E1', 'G4', 'E3', 'G5', 'E5'), []],
 [('E1', 'G4', 'E3', 'G5', 'F24'),
  [Ideal (4*c + (-i)*d + (-i + 2)*e + (i)*f, 5*d^2 + (8*i - 6)*d*e + (-24*i - 7)*e^2 + (-4*i - 2)*d*f + (36*i + 38)*e*f + (4*i

In [143]:
ideales = [_ for i in range(len(all_ideals))]
poly_es = [_ for i in range(len(all_ideals))]
e_sosts = [_ for i in range(len(all_ideals))]
poly_ds = [_ for i in range(len(all_ideals))]
d_sosts = [_ for i in range(len(all_ideals))]
for i in range(len(all_ideals)):
    ideales[i] = all_ideals[i]
    poly_es[i] = ideales[i].gens()[0]
    e_sosts[i] = {e: -(poly_es[i]-poly_es[i].coefficient(e)*e)/poly_es[i].coefficient(e)}
    poly_ds[i] = [gen for gen in ideales[i].subs(e_sosts[i]).gens() if gen !=0][0]
    d_sosts[i] = {d:-(poly_ds[i]-poly_ds[i].coefficient(d)*d)/poly_ds[i].coefficient(d)}
    print([gen.subs(e_sosts[i]).subs(d_sosts[i]) for gen in ideales[i].gens()])

[0, 0]
[0, 0]
[0, 0]
[0, 0]


In [160]:
SE1_0 = SE1.subs(e_sosts[0]).subs(d_sosts[0])

In [161]:
SE1_3 = SE1.subs(e_sosts[3]).subs(d_sosts[3]).subs({c:l, f:m})

In [170]:
vrs = P.gens()[0:4]
mon = (sum(vrs) ^ 3).monomials()
current_conds = list(set(matrix([[SE1_0.coefficient(mn) for mn in mon], [SE1_3.coefficient(mn) for mn in mon]]).minors(2)))
poly = P.ideal(current_conds).radical().primary_decomposition()[0].gens()[0]
poly.coefficient(l), poly.coefficient(m)

(5*c + (4*i + 3)*f, (-2*i + 1)*c + (2*i - 1)*f)

In [171]:
sost1 = {l: poly.coefficient(m), m: -poly.coefficient(l)}
SE1_3_new = SE1_3.subs(sost1)

In [172]:
SE1_0.are_cubics_same(SE1_3_new)

True

In [168]:
SE1_0

x^2*z*c^2 + (i)*x*y*z*c^2 - x*z^2*c^2 + (i - 1)*x*y*t*c^2 + 2*y^2*t*c^2 + (-i - 1)*y*z*t*c^2 + (-i)*y*t^2*c^2 + (i - 2)*x^2*y*c*f + (-i + 2)*x*y^2*c*f + 2*x^2*z*c*f + (-i - 3)*x*y*z*c*f + x*z^2*c*f + (-2*i)*x*y*t*c*f + (i - 1)*y^2*t*c*f + (i + 1)*y*z*t*c*f + (i)*y*t^2*c*f + (-i - 2)*x^2*y*f^2 + (i + 2)*x*y^2*f^2 + x^2*z*f^2 - x*y*z*f^2 + (i + 1)*x*y*t*f^2 + (-i - 1)*y^2*t*f^2

In [175]:
SE1_1 = SE1.subs(e_sosts[1]).subs(d_sosts[1]).subs({c:l, f:m})
SE1_1

x^2*z*l^2 + (-i)*x*y*z*l^2 - x*z^2*l^2 + (-i - 1)*x*y*t*l^2 + 2*y^2*t*l^2 + (i - 1)*y*z*t*l^2 + (i)*y*t^2*l^2 + (-i - 2)*x^2*y*l*m + (i + 2)*x*y^2*l*m + 2*x^2*z*l*m + (i - 3)*x*y*z*l*m + x*z^2*l*m + (2*i)*x*y*t*l*m + (-i - 1)*y^2*t*l*m + (-i + 1)*y*z*t*l*m + (-i)*y*t^2*l*m + (i - 2)*x^2*y*m^2 + (-i + 2)*x*y^2*m^2 + x^2*z*m^2 - x*y*z*m^2 + (-i + 1)*x*y*t*m^2 + (i - 1)*y^2*t*m^2

In [235]:
L_set_ideals = []
L_sets = list(set([get_permuted_L_set(perm) for perm in SE1_1.find_admissible_permutations()]))
vrs = P.gens()[0:4]
mon = (sum(vrs) ^ 3).monomials()
for L_set in L_sets:
    if L_sets.index(L_set)%10 == 0:
        print(L_sets.index(L_set))
    proj = SE1_1.find_projectivity(SE1_1.L_set_base, L_set)
    eqn = SE1_1.eqn.subs(change_coordinates(proj))
    eqn = list(eqn.factor())[-1][0]
    if eqn.degree(x) != 2:
        print("error")
    current_conds = list(set(matrix([[SE1_0.coefficient(mn) for mn in mon], [eqn.coefficient(mn) for mn in mon]]).minors(2)))
    prim_dec = P.ideal(current_conds).radical().primary_decomposition()
    L_set_ideals += [(L_set,ideal) for ideal in prim_dec if ideal.gens()==1]
L_set_ideals

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570


[]

In [16]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 3.28 s, sys: 301 ms, total: 3.58 s
Wall time: 24.6 s


576

In [17]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 1.72 s, sys: 229 ms, total: 1.95 s
Wall time: 8.91 s


4

In [18]:
lines_perms = []
for simm in simm_SE1_1:
    lines_perms.append(Permutation(SE1_1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(4, 'C4')

In [19]:
lines_perms_group.gens_small()

[(2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20)]

In [20]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46')]

In [22]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens() if perm.order()==4]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46'),
 ('E1', 'G4', 'F34', 'F16', 'E2', 'E6')]

In [28]:
lines_perms_group.gens()

[(),
 (2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20),
 (2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23),
 (2,22,19,3)(4,23,7,21)(5,6,25,26)(8,11,13,16)(9,17,14,12)(18,20,27,24)]